Nela, encontramos diversos tipos de informações que foram separadas em 4 grupos:

Informação demográfica - 3 variáveis
Doenças pré-existentes - 9 variáveis
Resultados do exame de sangue - 36 variáveis
Sinais vitais - 6 variáveis
Sabemos que há urgência na obtenção e manipulação de dados para melhorar a previsão e assim, conseguir preparar o sistema de saúde, evitando colapsos.

Nosso objetivo será prever quais pacientes precisarão ser admitidos na unidade de terapia intensiva e assim, definir qual a necessidade de leitos de UTI do hospital, a partir dos dados clínicos individuais disponíveis.

Quando conseguimos definir a quantidade de leitos necessários em um determinado hospital, conseguimos evitar rupturas, visto que, caso outra pessoa procure ajuda e, eventualmente, precise de cuidados intensivos, o modelo preditivo já conseguirá detectar essa necessidade e, desta forma, a remoção e transferência deste paciente pode ser organizada antecipadamente.

Queremos que você aplique tudo o que aprendeu durante toda sua trajetória no Bootcamp e construa um modelo com as técnicas de Machine Learning que busquem a nossa variável-resposta.

Tenha em mente que este projeto será apresentado, de maneira fictícia, para o gerente responsável pela modelagem de dados do time de Data Science do Hospital Sírio Libanês. Você precisará persuadi-lo de que seu modelo tem os pontos necessários para entrar em produção e ajudará a antever e evitar qualquer ruptura.

Como a entrega é obrigatória para certificação, montamos um conjunto de critérios mínimos para avaliação que vocês poderão usar como um guia para montar seu estudo.

Temos dois blocos a serem considerados:

Técnico
Prático
Na seção de critérios mínimos deste projeto, você encontra quais são os aspectos que compõem estes blocos e suas respectivas descrições.

Para que o seu projeto seja avaliado pelo Thiago G. Santos e Átila Iamarino, ao vivo, na live de revisão de projetos, submeta seu notebook ou a URL do seu projeto público no GitHub até dia 24/02 às 23h59.}

# Importar dados e bibliotecas

In [ ]:
import numpy as np
import pandas as pd

!pip install openpyxl

import openpyxl
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')
import os

pd.set_option('display.max_columns', 500)

In [ ]:
df = pd.read_excel('/kaggle/input/covid19/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx')

# TRATAR DADOS

In [ ]:
#Ordenar df for paciente e window
df.sort_values(['PATIENT_VISIT_IDENTIFIER','WINDOW'],inplace=True)

In [ ]:
#Algumas colunas booleanas estão como NaN ou -1. Vamos ajustar para 0 ou 1

adapt_bool = df[df.columns[df.nunique() == 1]]

adapt_bool = adapt_bool.fillna(0)
adapt_bool = adapt_bool.abs()

df[df.columns[df.nunique() == 1]] = adapt_bool

In [ ]:
#Paciente 199 não tem dados, vamos dropar
df.drop(df[df['PATIENT_VISIT_IDENTIFIER']==199].index,inplace=True)

### Colunas com NaN

In [ ]:
df[['PATIENT_VISIT_IDENTIFIER','ICU'] + list(df.columns[df.isna().any()])].head(10)

In [ ]:
class RepeatedSet:
    def __init__(self,d,l):
        self.d = d
        self.l = l

def repeated_columns(dataframe):
    
    """This function takes a DataFrame Object and return a dictionary with 
    all sets of columns that contains the same values. 
    
    The keys of the dictionary are the names of the column used to compare values """
    
    temp = dataframe.copy()
    repeated_sets={}

    for j in dataframe.columns:
        if j in temp.columns:
            repeats =[]
            for i in dataframe.columns: 
                repeats.append(temp[j].equals(dataframe[i]))
            if repeats.count(True) > 1:
                repeated_sets[j] = dataframe.columns[repeats].to_list()[1:]
            else:
                pass
            temp.drop(dataframe.columns[repeats].to_list(),axis = 1,inplace=True)
        else:
            pass
    return RepeatedSet(repeated_sets,[item for sublist in repeated_sets.values() for item in sublist])

In [ ]:
sets= repeated_columns(df)

In [ ]:
df.drop(sets.l,axis=1,inplace=True)

In [ ]:
#rename columns that represent dropped ones columns
column_maps = dict(zip(sets.d.keys(),[item[:item.rindex('_')] for item in sets.d.keys()]))

df.rename(columns=column_maps,inplace=True)

In [ ]:
#I dont want to use gender in this analysis
df.drop('GENDER',axis=1,inplace=True)

In [ ]:
#encode non numeric variables
window_dummies = pd.get_dummies(df['WINDOW'])

df =pd.concat([df,window_dummies],axis=1)
df.drop('WINDOW',axis=1,inplace=True)

df['AGE_PERCENTIL'] = df['AGE_PERCENTIL'].astype('category').cat.codes +1

In [ ]:
first_col = df.pop('ICU')

df.insert(0, 'ICU', first_col)

In [ ]:
df.head(20)

Analise dados clinicos

Analise dados Laboratoriais

Analise dados Monitoramento

# Modelo Iteration 1

Modelar sem indicadores de sanguineos ou de monitoramento

In [ ]:
feature_laboratorial = ['ALBUMIN',
 'BE_ARTERIAL',
 'BE_VENOUS',
 'BIC_ARTERIAL',
 'BIC_VENOUS',
 'BILLIRUBIN',
 'BLAST',
 'CALCIUM',
 'CREATININ',
 'FFA',
 'GGT',
 'GLUCOSE',
 'HEMATOCRITE',
 'HEMOGLOBIN',
 'INR',
 'LACTATE',
 'LEUKOCYTES',
 'LINFOCITOS',
 'NEUTROPHILES',
 'P02_ARTERIAL',
 'P02_VENOUS',
 'PC02_ARTERIAL',
 'PC02_VENOUS',
 'PCR',
 'PH_ARTERIAL',
 'PH_VENOUS',
 'PLATELETS',
 'POTASSIUM',
 'SAT02_ARTERIAL',
 'SAT02_VENOUS',
 'SODIUM',
 'TGO',
 'TGP',
 'TTPA',
 'UREA',
 'DIMER']

In [ ]:
feature_monitoramento = [ 'BLOODPRESSURE_DIASTOLIC_MEAN',
 'BLOODPRESSURE_SISTOLIC_MEAN',
 'HEART_RATE_MEAN',
 'RESPIRATORY_RATE_MEAN',
 'TEMPERATURE_MEAN',
 'OXYGEN_SATURATION_MEAN',
 'BLOODPRESSURE_DIASTOLIC_MEDIAN',
 'BLOODPRESSURE_SISTOLIC_MEDIAN',
 'HEART_RATE_MEDIAN',
 'RESPIRATORY_RATE_MEDIAN',
 'TEMPERATURE_MEDIAN',
 'OXYGEN_SATURATION_MEDIAN',
 'BLOODPRESSURE_DIASTOLIC_MIN',
 'BLOODPRESSURE_SISTOLIC_MIN',
 'HEART_RATE_MIN',
 'RESPIRATORY_RATE_MIN',
 'TEMPERATURE_MIN',
 'OXYGEN_SATURATION_MIN',
 'BLOODPRESSURE_DIASTOLIC_MAX',
 'BLOODPRESSURE_SISTOLIC_MAX',
 'HEART_RATE_MAX',
 'RESPIRATORY_RATE_MAX',
 'TEMPERATURE_MAX',
 'OXYGEN_SATURATION_MAX',
 'BLOODPRESSURE_DIASTOLIC_DIFF',
 'BLOODPRESSURE_SISTOLIC_DIFF',
 'HEART_RATE_DIFF',
 'RESPIRATORY_RATE_DIFF',
 'TEMPERATURE_DIFF',
 'OXYGEN_SATURATION_DIFF',
 'BLOODPRESSURE_DIASTOLIC_DIFF_REL',
 'BLOODPRESSURE_SISTOLIC_DIFF_REL',
 'HEART_RATE_DIFF_REL',
 'RESPIRATORY_RATE_DIFF_REL',
 'TEMPERATURE_DIFF_REL',
 'OXYGEN_SATURATION_DIFF_REL']

In [ ]:
X = ['AGE_ABOVE65',
 'AGE_PERCENTIL',
 'DISEASE GROUPING 1',
 'DISEASE GROUPING 2',
 'DISEASE GROUPING 3',
 'DISEASE GROUPING 4',
 'DISEASE GROUPING 5',
 'DISEASE GROUPING 6',
 'HTN',
 'IMMUNOCOMPROMISED',
 'OTHER',
 '0-2',
 '2-4',
 '4-6',
 '6-12',
 'ABOVE_12']

y = ['ICU']

In [ ]:
df[X]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[X], df[y], test_size=0.33, random_state=1111)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

In [ ]:
y_hat = clf.predict(X_test)

In [ ]:
y_test['y_hat'] = y_hat

In [ ]:
y_test